# Plan

- Implement the eight-point algorithm
- Compute the fundamental matrix $\mathrm F$ for a pair of real images using all correspondences
    - Optional: try to implement a [robust](https://ru.wikipedia.org/wiki/RANSAC) $\mathrm F$ estimator
- Visualize epipolar lines using the fundamental matrix

In [ ]:
import numpy as np
import torch
import matplotlib.pylab as plt


def dehomogenize_points(points):
    return points[..., :-1] / (points[..., -1:] + 1e-8 * torch.sign(points[..., -1:]))

def homogenize_points(points):
    last_coordinate = torch.ones_like(points[..., -1:])
    return torch.cat((points, last_coordinate), dim=-1)

# Eight-point Algorithm

In [ ]:
# synthetic data
points_3d = torch.randn(8, 4)
H_l = torch.randn(3, 4)
H_r = torch.randn(3, 4)

points_l = points_3d @ H_l.T
points_r = points_3d @ H_r.T

In [ ]:
# visualize the synthetic data
fig, ax = plt.subplots(1, 2)

ax[0].set_title('left')
ax[0].scatter(*torch.chunk(dehomogenize_points(points_l), 2, dim=-1))
ax[1].set_title('right')
ax[1].scatter(*torch.chunk(dehomogenize_points(points_r), 2, dim=-1))

In [ ]:
def get_fundamental_matrix(points_l, points_r):
    raise NotImplementedError
    
def compute_residuals(F, points_l, points_r):
    raise NotImplementedError

def compute_errors(F, points_l, points_r):
    # optional
    raise NotImplementedError

In [ ]:
F = get_fundamental_matrix(points_l, points_r)
residuals = compute_residuals(F, points_l, points_r)

print(residuals.abs().max())

# Real Example

In [ ]:
left = plt.imread('left.jpg')
right = plt.imread('right.jpg')
h, w, _ = left.shape  # same shape for left and right
# see https://docs.opencv.org/4.x/dc/dc3/tutorial_py_matcher.html for details
correspondences = torch.load('correspondences.pt')
# points are in homogeneous coordinates, however z = 1 for all points
points_l = correspondences['points_l']
points_r = correspondences['points_r']

In [ ]:
def plot_correspondences(ax, points_l, points_r, w):
    for point_l, point_r in zip(points_l, points_r):
        ax.plot([point_l[0], point_r[0] + w],
                [point_l[1], point_r[1]],
               marker='o')

In [ ]:
fig, ax = plt.subplots(figsize=(17, 10))
concatenated_image = np.concatenate((left, right), axis=1)
ax.imshow(concatenated_image)
plot_correspondences(ax, points_l, points_r, w)
ax.axis('off');

In [ ]:
F = get_fundamental_matrix(points_l,
                           points_r)
resiudals = compute_residuals(F, points_l, points_r)
plt.hist(residuals.abs());

In [ ]:
def plot_epipolar_lines(ax, F, point_l, point_r, w):
    # plot two epipolar lines for a single correspondence
    raise NotImplementedError

In [ ]:
fig, ax = plt.subplots(figsize=(17, 10))
ax.imshow(concatenated_image)
for point_l, point_r in zip(points_l, points_r):
    plot_epipolar_lines(ax, F, point_l, point_r, w)
ax.axis('off');
#plt.savefig('images_with_epipolar_lines.jpg', bbox_inches='tight', dpi=200)